In [1]:
%load_ext line_profiler
from adapter import mkcalc
import numpy as np
import pandas as pd
from scipy.optimize import fsolve
import matplotlib.pyplot as plt
import sys
import time

### Solving original approximation. Using default parameters provided at *al_x.py*: no bgs and 20% of alpha due to weak selection

In [2]:
def analyticalTime(bgs=0.999,al=0.2,alweak=0.2,gam=-83,popsize=500,sample=250,lCoding=501,lNonCoding=10**6,gamWeak=10,gamStrong=500):
    adap = mkcalc.AsympMK(B=bgs,gam_neg=gam,theta_f=0.001,alLow=alweak,alTot=al,neut_mid=False,L_mid=lCoding,Lf=lNonCoding,N=popsize,nsim=2500,pref="unc",gL=10,n=sample,gH=500)
    
    adap.set_theta_f()
    theta_f = adap.theta_f
    adap.B = 0.999
    adap.set_theta_f()
    adap.setPpos()
    adap.theta_f = theta_f
    adap.B = bgs

    pos = adap.alx(adap.gL,adap.gH,adap.pposL,adap.pposH)
    nopos = adap.alx_nopos(adap.gL,adap.gH,adap.pposL,adap.pposH)

In [3]:
%timeit analyticalTime()

3.58 s ± 69.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [4]:
%lprun -f analyticalTime analyticalTime()

Timer unit: 1e-06 s

Total time: 11.3068 s
File: <ipython-input-2-dbf5972f16d0>
Function: analyticalTime at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def analyticalTime(bgs=0.999,al=0.2,alweak=0.2,gam=-83,popsize=500,sample=250,lCoding=501,lNonCoding=10**6,gamWeak=10,gamStrong=500):
     2         1         95.0     95.0      0.0      adap = mkcalc.AsympMK(B=bgs,gam_neg=gam,theta_f=0.001,alLow=alweak,alTot=al,neut_mid=False,L_mid=lCoding,Lf=lNonCoding,N=popsize,nsim=2500,pref="unc",gL=10,n=sample,gH=500)
     3                                               
     4         1        409.0    409.0      0.0      adap.set_theta_f()
     5         1          3.0      3.0      0.0      theta_f = adap.theta_f
     6         1          2.0      2.0      0.0      adap.B = 0.999
     7         1        350.0    350.0      0.0      adap.set_theta_f()
     8         1     601476.0 601476.0      5.3      adap.setPpos()
  

**In order to get the alpha(x) and the expected values of each category (Dn, Ds, Pn, Ps) we got a mean of 3.58s per iteration when using Ne=500 n=250**. ~90% of computation time is dediacted to estimate alx. In addition, I changed the original values to show which are that most influence on computing time. Since the expectation is calculated for each value of the frequency spectrum, population and sample size must be the limiting variables in the process. Changes on gamma's values, coding length and non-coding length seems not to influence in computation time drastically.

#### Changing gamma values

In [12]:
%timeit analyticalTime(bgs=0.999,al=0.2,alweak=0.2,gam=-200,popsize=500,sample=250,lCoding=501,lNonCoding=10**6,gamWeak=10,gamStrong=500)
%timeit analyticalTime(bgs=0.999,al=0.2,alweak=0.2,gam=-83,popsize=500,sample=250,lCoding=501,lNonCoding=10**6,gamWeak=20,gamStrong=500)
%timeit analyticalTime(bgs=0.999,al=0.2,alweak=0.2,gam=-83,popsize=500,sample=250,lCoding=501,lNonCoding=10**6,gamWeak=10,gamStrong=400)
%timeit analyticalTime(bgs=0.999,al=0.2,alweak=0.2,gam=-200,popsize=500,sample=250,lCoding=501,lNonCoding=10**6,gamWeak=40,gamStrong=600)

3.56 s ± 39 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
3.55 s ± 6.94 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
3.65 s ± 43.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
3.62 s ± 12.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


#### Reducing Ne in a half

In [13]:
%timeit analyticalTime(bgs=0.999,al=0.2,alweak=0.2,gam=-83,popsize=250,sample=250,lCoding=501,lNonCoding=10**6)

2.07 s ± 46.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


#### Reducing n in a half

In [14]:
%timeit analyticalTime(bgs=0.999,al=0.2,alweak=0.2,gam=-83,popsize=500,sample=175,lCoding=501,lNonCoding=10**6)

3.19 s ± 80.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


#### Duplicating Ne and n values

In [15]:
%timeit analyticalTime(bgs=0.999,al=0.2,alweak=0.2,gam=-83,popsize=1000,sample=500,lCoding=501,lNonCoding=10**6)

8.53 s ± 126 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


#### Reducing length in a half

In [16]:
%timeit analyticalTime(bgs=0.999,al=0.2,alweak=0.2,gam=-83,popsize=500,sample=250,lCoding=250,lNonCoding=5*10**5)

3.47 s ± 45 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


#### Duplicating length values

In [17]:
%timeit analyticalTime(bgs=0.999,al=0.2,alweak=0.2,gam=-83,popsize=500,sample=250,lCoding=1001,lNonCoding=2*10**6)

3.46 s ± 36.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


**setPpos and alpha calculations are the most problematic functions. Both need almost a second to run with Ne=500 and n=250. I compiled the functions manually with originals values to run step-by-step in order to check where are the bottlenecks*

### Exploring computation times on *set_theta_f* and *alx* functions manually. mkcalcManual contains the functions loaded with the values processed before execute *pos=alx(...)*

In [20]:
%load_ext line_profiler
import numpy as np
import pandas as pd
from scipy.optimize import fsolve
import sys
import time
import inspect

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [22]:
import sys
sys.path.insert(0, '/home/jmurga/mktest/scripts/')
from mkcalcManual import *

In [23]:
%lprun -f set_theta_f_manual set_theta_f_manual()

Timer unit: 1e-06 s

Total time: 0.00029 s
File: /home/jmurga/mktest/scripts/mkcalcManual.py
Function: set_theta_f_manual at line 52

Line #      Hits         Time  Per Hit   % Time  Line Contents
    52                                           def set_theta_f_manual():
    53         1        288.0    288.0     99.3  	theta_f  = fsolve(lambda theta: Br_manual(Lf,theta)-B,0.00001)
    54         1          2.0      2.0      0.7  	theta_f = theta_f[0]

**I think *set_theta_f* could not be optimize. It's just solving an equation with *x* parameters, in that sense only depends on *fsolve* computation time.**

In [24]:
print(inspect.getsource(set_theta_f_manual))

def set_theta_f_manual():
	theta_f  = fsolve(lambda theta: Br_manual(Lf,theta)-B,0.00001)
	theta_f = theta_f[0]



In [25]:
print(inspect.getsource(Br_manual))

def Br_manual(Lmax,theta):
	t = -1.*gam_neg/(NN+0.)
	u = theta/(2.*NN)
	r = rho/(2.*NN)
	return np.exp(-4*u*Lmax/(2*Lmax*r+t))



In [26]:
print(inspect.getsource(alx_manual))

def alx_manual(gammaL,gammaH,pposL,pposH):
	ret = []

	#Fixation
	fN = B*fixNeut_manual()
	fNeg = B*fixNegB_manual(0.5*pposH+0.5*pposL)
	fPosL = fixPosSim_manual(gammaL,0.5*pposL)
	fPosH = fixPosSim_manual(gammaH,0.5*pposH)

	#Pol
	neut = cumuSfs_manual(DiscSFSNeutDown_manual())
	selH = cumuSfs_manual(DiscSFSSelPosDown_manual(gammaH,pposH))
	selL = cumuSfs_manual(DiscSFSSelPosDown_manual(gammaL,pposL))
	selN = cumuSfs_manual(DiscSFSSelNegDown_manual(pposH+pposL))
	
	sel = []
	for i in range(0,len(selH)):
		sel.append((selH[i]+selL[i])+selN[i])
	for i in range(0,nn-1):
		ret.append(float(1. - (fN/(fPosL + fPosH+  fNeg+0.))* sel[i]/neut[i]))
	return (ret,sel,neut,selH,selL,selN,fN,fNeg,fPosL,fPosH)



In [27]:
%lprun -f alx_manual alx_manual(gL,gH,pposL,pposH)

Timer unit: 1e-06 s

Total time: 5.88841 s
File: /home/jmurga/mktest/scripts/mkcalcManual.py
Function: alx_manual at line 163

Line #      Hits         Time  Per Hit   % Time  Line Contents
   163                                           def alx_manual(gammaL,gammaH,pposL,pposH):
   164         1          1.0      1.0      0.0  	ret = []
   165                                           
   166                                           	#Fixation
   167         1          3.0      3.0      0.0  	fN = B*fixNeut_manual()
   168         1       5783.0   5783.0      0.1  	fNeg = B*fixNegB_manual(0.5*pposH+0.5*pposL)
   169         1        305.0    305.0      0.0  	fPosL = fixPosSim_manual(gammaL,0.5*pposL)
   170         1       9808.0   9808.0      0.2  	fPosH = fixPosSim_manual(gammaH,0.5*pposH)
   171                                           
   172                                           	#Pol
   173         1     272688.0 272688.0      4.6  	neut = cumuSfs_manual(DiscSFSNeutDown_m

In [28]:
%timeit fN2 = B*fixNeut_manual()
%timeit fNeg2 = B*fixNegB_manual(0.5*pposH+0.5*pposL)
%timeit fPosL2 = fixPosSim_manual(gL,0.5*pposL)
%timeit fPosH2 = fixPosSim_manual(gH,0.5*pposH)
%timeit neut2 = cumuSfs_manual(DiscSFSNeutDown_manual())
%timeit selH2 = cumuSfs_manual(DiscSFSSelPosDown_manual(gH,pposH))
%timeit selL2 = cumuSfs_manual(DiscSFSSelPosDown_manual(gL,pposL))
%timeit selN2 = cumuSfs_manual(DiscSFSSelNegDown_manual(pposH+pposL))

143 ns ± 0.555 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)
935 µs ± 29.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
81.3 µs ± 1.62 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
1.57 ms ± 20.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
237 ms ± 1.81 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
277 ms ± 5.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
284 ms ± 12.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.16 s ± 10.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [29]:
print(inspect.getsource(DiscSFSSelNegDown_manual))

def DiscSFSSelNegDown_manual(ppos):
		return B*(theta_mid_neutral)*0.745*(np.dot(binomOp_manual(),DiscSFSSelNeg_manual(ppos)))[1:-1]



In [30]:
print(inspect.getsource(DiscSFSSelNeg_manual))

def DiscSFSSelNeg_manual(ppos):
	NN2 = int(round(NN*B))
	dFunc = np.vectorize(FullNeg_manual)
	return np.multiply(1./(NN2+0.),dFunc(ppos,[i/(NN2+0.) for i in range(0,NN2+1)]))



In [31]:
print(inspect.getsource(FullNeg_manual))

def FullNeg_manual( ppos, x):
	beta = be/(1.*B)
	if x > 0 and x < 1.:
		return (1.-ppos)*(2.**-al)*(beta**al)*(-mpmath.zeta(al,x+beta/2.) + mpmath.zeta(al,(2+beta)/2.))/((-1.+x)*x)

	return 0.



Although it is vectorice each computation solving *selN* on negative gamma distribution is quite slow. The execution over the total array is about a second, and only one iteration is about 1ms

In [3]:
%timeit DiscSFSSelNegDown_manual(pposH+pposL)

1.17 s ± 6.32 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
NN2 = int(round(NN*B))
dFuncVector = np.vectorize(FullNeg_manual)

In [6]:
%timeit y = dFuncVector(pposH+pposL,[i/(NN2+0.) for i in range(0,NN2+1)])

1.02 s ± 25.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Checking one iteration time

In [20]:
x = [i/(NN2+0.) for i in range(0,NN2+1)]

In [22]:
%timeit  b = FullNeg_manual(pposH+pposL,x[1])

1.23 ms ± 44.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


### Solving *selN* with Julia

In [41]:
from adapter_dev import mkcalc_dev # Including scripts solving equations with Julia
from adapter import mkcalc

In [42]:
adap = mkcalc.AsympMK(B=0.999,gam_neg=-83,theta_f=0.001,alLow=0.2,alTot=0.2,neut_mid=False,L_mid=501,Lf=10**6,N=500,nsim=2500,pref="unc",gL=10,n=250,gH=500)
adap.set_theta_f()
theta_f = adap.theta_f
adap.setPpos()
adap.theta_f = theta_f
adap.B = 0.999

In [43]:
adap_dev = mkcalc_dev.AsympMK(B=0.999,gam_neg=-83,theta_f=0.001,alLow=0.2,alTot=0.2,neut_mid=False,L_mid=501,Lf=10**6,N=500,nsim=2500,pref="unc",gL=10,n=250,gH=500)
adap_dev.set_theta_f()
theta_f = adap_dev.theta_f
adap_dev.setPpos()
adap_dev.theta_f = theta_f
adap_dev.B = 0.999

In [44]:
%timeit adap.DiscSFSSelNegDown(adap.pposH+adap.pposL)

1.13 s ± 8.18 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [40]:
%timeit adap_dev.DiscSFSSelNegDown(adap_dev.pposH+adap_dev.pposL)

238 ms ± 8.59 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
